### Run this cell for Google colab only

In [5]:
!git clone https://github.com/chang1331/DeblurGAN
import os
os.chdir("DeblurGAN/")
%tensorflow_version 1.x
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Cloning into 'DeblurGAN'...
remote: Enumerating objects: 155, done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (155/155), 152.10 MiB | 12.30 MiB/s, done.
Resolving deltas: 100% (19/19), done.


#### Importing Libraries

In [0]:
import numpy as np
import glob, cv2, os
import datetime
import tqdm

#from deblurgan.utils import write_log
from losses import wasserstein_loss, perceptual_loss
from model import generator_model, discriminator_model, generator_containing_discriminator_multiple_outputs

from keras.callbacks import TensorBoard
from keras.optimizers import Adam
#for saving weights
BASE_DIR = 'weights/'


In [0]:
# Read data from numpy file
def load_data(path):
    data = np.load(path)
    data = (data - 0.5) / 0.5
    return data

# Saving weights after each iteration
def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join(BASE_DIR, '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)

# Training of model
def train_multiple_outputs(path1, path2, batch_size, epoch_num, critic_updates=5):
    x_train = load_data(path1)
    y_train = load_data(path2)

    g = generator_model(input_shape)
    d = discriminator_model(input_shape)
    d_on_g = generator_containing_discriminator_multiple_outputs(g, d, input_shape)

    d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    d.trainable = True
    d.compile(optimizer=d_opt, loss=wasserstein_loss)
    d.trainable = False
    loss = [perceptual_loss, wasserstein_loss]
    loss_weights = [100, 1]
    d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
    d.trainable = True

    output_true_batch, output_false_batch = np.ones((batch_size, 1)), -np.ones((batch_size, 1))

    tensorboard_callback = TensorBoard("../")
    print("returning")

    for epoch in tqdm.tqdm(range(epoch_num)):
        permutated_indexes = np.random.permutation(x_train.shape[0])

        d_losses = []
        d_on_g_losses = []
        for index in range(int(x_train.shape[0] / batch_size)):
            batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
            image_blur_batch = x_train[batch_indexes]
            image_full_batch = y_train[batch_indexes]

            generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

            for _ in range(critic_updates):
                d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
                d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
                d_losses.append(d_loss)

            d.trainable = False

            d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])
            d_on_g_losses.append(d_on_g_loss)

            d.trainable = True

        # write_log(tensorboard_callback, ['g_loss', 'd_on_g_loss'], [np.mean(d_losses), np.mean(d_on_g_losses)], epoch_num)
        print(np.mean(d_losses), np.mean(d_on_g_losses))
        with open('log.txt', 'a+') as f:
            f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

        save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))
    return


#### copy images

In [0]:
from google.colab import drive
import os

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r '/content/drive/My Drive/Colab Notebooks/Images/predicted.npy' '/content/Deblur_GAN/train_data_npy/'
!cp -r '/content/drive/My Drive/Colab Notebooks/Images/X_train_fit_I.npy' '/content/Deblur_GAN/train_data_npy/'

#### Set Parameters accordingly

In [0]:
# Setting parameters for our model
input_shape = (112,128,3)
batch_size = 32
epochs = 50
blurred_input_directory = "train_data_npy/predicted.npy"
unblurred_input_directory = "train_data_npy/X_train_fit_I.npy"

#### Training of model with the parameters provided above

In [0]:
train_multiple_outputs(blurred_input_directory, unblurred_input_directory, batch_size, epochs)

58892288/58889256 [==============================] - 5s 0us/step


  0%|          | 0/50 [00:00<?, ?it/s]

returning
-0.48922630730915323 1571.497


  4%|▍         | 2/50 [21:06<8:31:18, 639.13s/it]

-0.49991909649598504 1461.1819
-0.49990039574083434 1426.4456


...

-0.5 1216.32
-0.5 1213.544


 98%|█████████▊| 49/50 [8:22:04<10:11, 611.34s/it]

-0.5 1212.285


100%|██████████| 50/50 [8:32:16<00:00, 611.37s/it]

-0.5 1211.3942
